# Fake news Detection

### Importing required library

In [33]:
import pandas as pd 
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

### Inserting fake and real dataset

In [34]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [35]:
df_true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


Inserting a column called "class" for fake and real news dataset to categories fake and true news. 

In [36]:
df_fake["class"] = 0
df_true["class"] = 1

Removing last 10 rows from both the dataset, for manual testing  

In [37]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [38]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

Merging the manual testing dataframe in single dataset and save it in a csv file

In [39]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
#chown ja:admin afc_east.csv
df_manual_testing.to_csv("manual_testing.csv")

Merging the main fake and true dataframe

In [40]:
df_marge = pd.concat([df_fake, df_true], axis =0 )
df_marge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [41]:
df_marge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [42]:
#"title",  "subject" and "date" columns is not required for detecting the fake news, so drop the columns.

In [43]:
df = df_marge.drop(["title", "subject","date"], axis = 1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


#### Randomly shuffling the dataframe 

In [44]:
df = df.sample(frac = 1)

In [45]:
df.head(10)

,text,class
747,Donald Trump is desperate to distract everyone...,0
9272,The DOJ is reporting that they basically took ...,0
20570,Perhaps a little trigger warning therapy is ...,0
19272,DHAKA (Reuters) - Two journalists from Myanmar...,1
19356,"FLORENCE, Italy (Reuters) - Prime Minister The...",1
22106,Mark Anderson AFPWith a little help from the V...,0
19018,Liberal Delaware Sen. Chris Coons caused a sti...,0
14127,(Reuters) - South Korean President Moon Jae-in...,1
11708,,0
5320,WASHINGTON (Reuters) - President Donald Trump’...,1


In [46]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [47]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [48]:
df.columns

Index(['text', 'class'], dtype='object')

In [49]:
df.head()

,text,class
0,Donald Trump is desperate to distract everyone...,0
1,The DOJ is reporting that they basically took ...,0
2,Perhaps a little trigger warning therapy is ...,0
3,DHAKA (Reuters) - Two journalists from Myanmar...,1
4,"FLORENCE, Italy (Reuters) - Prime Minister The...",1


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [50]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [51]:
df["text"] = df["text"].apply(wordopt)
df.head(10)

,text,class
0,donald trump is desperate to distract everyone...,0
1,the doj is reporting that they basically took ...,0
2,perhaps a little trigger warning therapy is ...,0
3,dhaka reuters two journalists from myanmar...,1
4,florence italy reuters prime minister the...,1
5,mark anderson afpwith a little help from the v...,0
6,liberal delaware sen chris coons caused a sti...,0
7,reuters south korean president moon jae in...,1
8,,0
9,washington reuters president donald trump ...,1


#### Defining dependent and independent variable as x and y

In [52]:
x = df["text"]
y = df["class"]

#### Splitting the dataset into training set and testing set. 

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

#### Convert text to vectors

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### 1. Logistic Regression

In [56]:
from sklearn.linear_model import LogisticRegression

In [57]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [58]:
LR.score(xv_test, y_test)

0.9858288770053476

In [59]:
pred_lr=LR.predict(xv_test)

In [60]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5884
           1       0.98      0.99      0.99      5336

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### 2. Decision Tree Classification

### 3. Gradient Boosting Classifier

### 4. Random Forest Classifier

In [61]:
import pickle
filename = 'finalised_model.pkl'
pickle.dump(LR,open(filename,'wb'))

In [62]:
filename = 'vectorizer.pkl'
pickle.dump(vectorization,open(filename,'wb'))

# Model Testing With Manual Entry

### News

In [63]:
def output_lable(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "True News"
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    #pred_DT = DT.predict(new_xv_test)
    #pred_GBC = GBC.predict(new_xv_test)
    #pred_RFC = RFC.predict(new_xv_test)
    return print("\n\nLR Prediction: {}".format(output_lable(pred_LR[0])))

In [64]:
news = str(input())
manual_testing(news)

WASHINGTON (Reuters) - U.S. Representative Cathy McMorris Rodgers, the fourth-ranking Republican in the House of Representatives, on Friday told Fox News Channel that fellow Republican Representative Blake Farenthold should resign if an ethics investigation finds that sexual misconduct accusations against him have merit.  â€œIf these allegations are proven to be true ... I would hope that he would step aside,â€ said McMorris Rodgers, chair of the House Republican Conference. Farentholdâ€™s office was not immediately available to comment on her remarks. The House Ethics Committee said on Thursday it was investigating Farenthold, 55, over allegations of sexual harassment, discrimination and retaliation involving a former female staff member. It said it was also looking into whether the Corpus Christi, Texas, congressman made inappropriate statements to other members of his staff. Farenthold said on Thursday he was relieved that the ethics panel was going to look into the accusations. â€